## Data Wrangling

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import func, desc
from matplotlib.ticker import NullFormatter
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import seaborn as sns
from flask import Flask, jsonify
import datetime as dt

In [3]:
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite", echo=False)

In [4]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['otu', 'samples', 'samples_metadata']

In [5]:
Otu = Base.classes.otu

In [6]:
Samples = Base.classes.samples

In [7]:
Samples_MD = Base.classes.samples_metadata

In [8]:
session = Session(engine)

In [17]:
samples_query = session.query(Samples)
samples = pd.read_sql(samples_query.statement, samples_query.session.bind)

### OTU Table

In [9]:
otu_id_list = session.query(Otu.otu_id).all()
len(otu_id_list)

3674

In [10]:
otu_taxonomic_list = session.query(Otu.lowest_taxonomic_unit_found).all()
len(otu_taxonomic_list)

3674

In [11]:
otu_id = pd.DataFrame(otu_id_list)

In [12]:
otu_taxonomic = pd.DataFrame(otu_taxonomic_list)

In [13]:
otu_df = otu_id.join(otu_taxonomic)
otu_df.head()

,otu_id,lowest_taxonomic_unit_found
0,1,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
1,2,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
2,3,Archaea;Euryarchaeota;Halobacteria;Halobacteri...
3,4,Archaea;Euryarchaeota;Methanobacteria;Methanob...
4,5,Archaea;Euryarchaeota;Methanobacteria;Methanob...


In [37]:
new_otu_df = otu_df.groupby('lowest_taxonomic_unit_found').count().sort_values(by=['otu_id'], ascending=False).reset_index()
final_new_otu_df = new_otu_df.rename(columns={'otu_id' : "count"})
final_new_otu_df.head(10)

,lowest_taxonomic_unit_found,count
0,Bacteria,513
1,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,359
2,Bacteria;Actinobacteria;Actinobacteria;Actinom...,279
3,Bacteria;Firmicutes;Clostridia;Clostridiales,205
4,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,133
5,Bacteria;Bacteroidetes;Bacteroidia;Bacteroidal...,128
6,Bacteria;Firmicutes,111
7,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,109
8,Bacteria;Firmicutes;Bacilli,105
9,Bacteria;Firmicutes;Clostridia;Clostridiales;I...,79


In [57]:
otu_id_df = otu_df.groupby('lowest_taxonomic_unit_found').max().reset_index()
final_otu_df = final_new_otu_df.merge(otu_id_df, how='inner', on='lowest_taxonomic_unit_found')

In [43]:
named_otu_pie = list(final_otu_df[:10]['count'].values)

final_named_otu_pie = []
for i in named_otu_pie:
    final_named_otu_pie.append(int(i))

In [54]:
def get_otu_pie_labels():
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import sqlalchemy
    from sqlalchemy.ext.automap import automap_base
    from sqlalchemy.orm import Session
    from sqlalchemy import create_engine, inspect
    from sqlalchemy import func, desc
    from matplotlib.ticker import NullFormatter
    import matplotlib.dates as mdates
    from datetime import datetime, timedelta
    import seaborn as sns
    from flask import Flask, jsonify
    import datetime as dt
    engine = create_engine("sqlite:///belly_button_biodiversity.sqlite", echo=False)
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    Otu = Base.classes.otu
    session = Session(engine)
    otu_id_list = session.query(Otu.otu_id).all()
    otu_taxonomic_list = session.query(Otu.lowest_taxonomic_unit_found).all()
    otu_id = pd.DataFrame(otu_id_list)
    otu_taxonomic = pd.DataFrame(otu_taxonomic_list)
    otu_df = otu_id.join(otu_taxonomic)
    new_otu_df = otu_df.groupby('lowest_taxonomic_unit_found').count().sort_values(by=['otu_id'], ascending=False).reset_index()
    final_new_otu_df = new_otu_df.rename(columns={'otu_id' : "count"})
    otu_id_df = otu_df.groupby('lowest_taxonomic_unit_found').max().reset_index()
    final_otu_df = final_new_otu_df.merge(otu_id_df, how='inner', on='lowest_taxonomic_unit_found')
    named_labels = list(final_otu_df[:10]['otu_id'])
    return named_labels

In [17]:
def get_otu_pie_values():
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import sqlalchemy
    from sqlalchemy.ext.automap import automap_base
    from sqlalchemy.orm import Session
    from sqlalchemy import create_engine, inspect
    from sqlalchemy import func, desc
    from matplotlib.ticker import NullFormatter
    import matplotlib.dates as mdates
    from datetime import datetime, timedelta
    import seaborn as sns
    from flask import Flask, jsonify
    import datetime as dt
    engine = create_engine("sqlite:///belly_button_biodiversity.sqlite", echo=False)
    Base = automap_base()
    Base.prepare(engine, reflect=True)
    Otu = Base.classes.otu
    session = Session(engine)
    otu_id_list = session.query(Otu.otu_id).all()
    otu_taxonomic_list = session.query(Otu.lowest_taxonomic_unit_found).all()
    otu_id = pd.DataFrame(otu_id_list)
    otu_taxonomic = pd.DataFrame(otu_taxonomic_list)
    otu_df = otu_id.join(otu_taxonomic)
    new_otu_df = otu_df.groupby('lowest_taxonomic_unit_found').count().sort_values(by=['otu_id'], ascending=False).reset_index()
    named_otu_pie = list(new_otu_df[:10]['otu_id'].values)
    
    final_named_otu_pie = []
    for i in named_otu_pie:
        final_named_otu_pie.append(int(i))
        
    return final_named_otu_pie

### Samples Table

In [68]:
samples_df = pd.read_csv('belly_button_biodiversity_samples.csv')

In [66]:
def get_samples():
    import pandas as pd
    samples_df = pd.read_csv('belly_button_biodiversity_samples.csv')
    return list(samples_df.columns[1:])

In [69]:
get_samples()

['BB_940',
 'BB_941',
 'BB_943',
 'BB_944',
 'BB_945',
 'BB_946',
 'BB_947',
 'BB_948',
 'BB_949',
 'BB_950',
 'BB_952',
 'BB_953',
 'BB_954',
 'BB_955',
 'BB_956',
 'BB_958',
 'BB_959',
 'BB_960',
 'BB_961',
 'BB_962',
 'BB_963',
 'BB_964',
 'BB_966',
 'BB_967',
 'BB_968',
 'BB_969',
 'BB_970',
 'BB_971',
 'BB_972',
 'BB_973',
 'BB_974',
 'BB_975',
 'BB_978',
 'BB_1233',
 'BB_1234',
 'BB_1235',
 'BB_1236',
 'BB_1237',
 'BB_1238',
 'BB_1242',
 'BB_1243',
 'BB_1246',
 'BB_1253',
 'BB_1254',
 'BB_1258',
 'BB_1259',
 'BB_1260',
 'BB_1264',
 'BB_1265',
 'BB_1273',
 'BB_1275',
 'BB_1276',
 'BB_1277',
 'BB_1278',
 'BB_1279',
 'BB_1280',
 'BB_1281',
 'BB_1282',
 'BB_1283',
 'BB_1284',
 'BB_1285',
 'BB_1286',
 'BB_1287',
 'BB_1288',
 'BB_1289',
 'BB_1290',
 'BB_1291',
 'BB_1292',
 'BB_1293',
 'BB_1294',
 'BB_1295',
 'BB_1296',
 'BB_1297',
 'BB_1298',
 'BB_1308',
 'BB_1309',
 'BB_1310',
 'BB_1374',
 'BB_1415',
 'BB_1439',
 'BB_1441',
 'BB_1443',
 'BB_1486',
 'BB_1487',
 'BB_1489',
 'BB_1490',
 

### Samples Metadata

In [19]:
samples_meta_df = pd.read_csv('Belly_Button_Biodiversity_Metadata.csv')
samples_meta_df.head()

,SAMPLEID,EVENT,ETHNICITY,GENDER,AGE,WFREQ,BBTYPE,LOCATION,COUNTRY012,ZIP012,...,DOG,CAT,IMPSURFACE013,NPP013,MMAXTEMP013,PFC013,IMPSURFACE1319,NPP1319,MMAXTEMP1319,PFC1319
0,940,BellyButtonsScienceOnline,Caucasian,F,24.0,2.0,I,Beaufort/NC,usa,22306,...,no,no,8852.0,37.172222,54.5,NaN,1,NaN,33.990002,25.5
1,941,NaN,Caucasian/Midleastern,F,34.0,1.0,I,Chicago/IL,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,943,BellyButtonsScienceOnline,Caucasian,F,49.0,1.0,I,Omaha/NE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,944,BellyButtonsScienceOnline,European,M,44.0,1.0,I,NewHaven/CT,usa,7079,...,no,yes,NaN,35.816666,16.0,NaN,0,6567.0,32.403332,28.5
4,945,BellyButtonsScienceOnline,Caucasian,F,48.0,1.0,I,Philidelphia/PA,usa,84404,...,no,no,NaN,37.783333,4.0,NaN,0,5613.0,33.634445,24.0


In [20]:
def get_samples_metadata_columns():
    import pandas as pd
    samples_meta_df = pd.read_csv('Belly_Button_Biodiversity_Metadata.csv')
    return list(samples_meta_df.columns)

In [21]:
def get_samples_metadata_values():
    import pandas as pd
    samples_meta_df = pd.read_csv('Belly_Button_Biodiversity_Metadata.csv')
    return samples_meta_df.values.tolist()